In [ ]:
from google.colab import drive
from datetime import datetime
import yfinance as yf
import scipy.special
import pandas as pd
import numpy as np

In [ ]:
start = datetime(2000, 1, 1)
end = datetime(2024, 8, 31)
tckr = '^GSPC'
ticker = yf.Ticker(tckr)
ticker_historical = ticker.history(start=start, end=end, interval="1d")

In [ ]:
window_size = 21
high = ticker_historical.High
low = ticker_historical.Low
open = ticker_historical.Open
close = ticker_historical.Close

In [ ]:
def HistoricalVolatility(close, window_size):
  log_returns = (np.log(close / close.shift(1))).dropna()
  HV = log_returns.rolling(window=window_size).std().dropna()

  return HV

In [ ]:
# def CloseToCloseVolatility(open, close, window_size):
#   log_returns = ((np.log(open / close.shift(1))) + (np.log(close / open))).dropna()
#   CCV = log_returns.rolling(window=window_size).std().dropna()

#   return CCV

In [ ]:
def CloseToCloseVolatility(open, close, window_size):
  log_returns = (np.log(open / close.shift(1)) + np.log(close / open)).dropna()
  mean_log_returns = log_returns.mean()
  squared_deviations = (log_returns - mean_log_returns) ** 2
  CCV = squared_deviations.rolling(window=window_size).mean().dropna()
  CCV = np.sqrt(CCV)

  return CCV

In [ ]:
def ParkinsonVolatility(high, low, window_size):
  constant_term = (1 / (4 * np.log(2)))
  log_returns = (np.log(high / low)).dropna()
  squared_returns = log_returns ** 2
  mean_returns = squared_returns.rolling(window=window_size).mean().dropna()
  PV = np.sqrt(constant_term * mean_returns)

  return PV

In [ ]:
def GarmanKlassVolatility(high, low, open, close, window_size):
  log_returns_term1 = (np.log(high / low)).dropna()
  log_returns_term2 = (np.log(close / open)).dropna()
  constant_term = (2 * np.log(2) -1)
  squared_returns = (log_returns_term1 ** 2) - (constant_term * (log_returns_term2 ** 2))
  mean_returns = squared_returns.rolling(window=window_size).mean().dropna()
  GKV = np.sqrt(mean_returns/2)

  return GKV

In [ ]:
def RogersSatchellVolatility(high, low, open, close, window_size):
  log_returns_term1 = (np.log(high / close)) * (np.log(high / open))
  log_returns_term2 = (np.log(low / close)) * (np.log(low / open))
  log_returns = (log_returns_term1 + log_returns_term2).dropna()
  mean_returns = log_returns.rolling(window=window_size).mean().dropna()
  RSV = np.sqrt(mean_returns)

  return RSV

In [ ]:
def YangZhangVolatility(high, low, open, close, window_size):
  log_returns = (np.log(open / close.shift(1))).dropna()
  mean_returns = log_returns.rolling(window=window_size).mean().dropna()
  squared_returns = (log_returns - mean_returns) ** 2
  OV = squared_returns.rolling(window=window_size).mean().dropna()

  log_returns = (np.log(close / open)).dropna()
  mean_returns = log_returns.rolling(window=window_size).mean().dropna()
  squared_returns = (log_returns - mean_returns) ** 2
  CV = squared_returns.rolling(window=window_size).mean().dropna()

  RSV = RogersSatchellVolatility(high, low, open, close, window_size)
  k = 0.34 / (1.34 + ((window_size + 1)/(window_size -1)))

  YZV = (np.sqrt(OV + (k*CV) + ((1-k)*RSV))).dropna()

  return YZV

In [ ]:
def GarmanKlassYangZhangVolatility(high, low, open, close, window_size):
  log_returns_term1 = (np.log(open / close.shift(1))).dropna()
  log_returns_term2 = (np.log(high / low)).dropna()
  log_returns_term3 = (np.log(close / open)).dropna()
  constant_term = (2 * np.log(2) -1)
  squared_returns = ((log_returns_term1 ** 2) / 2)  + ((log_returns_term2 ** 2) / 2) - (constant_term * (log_returns_term3 ** 2))
  mean_returns = squared_returns.rolling(window=window_size).mean().dropna()
  GKYZV = np.sqrt(mean_returns)

  return GKYZV

In [ ]:
HV = HistoricalVolatility(close, window_size)
CCV = CloseToCloseVolatility(open, close, window_size)
PV = ParkinsonVolatility(high, low, window_size)
GKV = GarmanKlassVolatility(high, low, open, close, window_size)
RSV = RogersSatchellVolatility(high, low, open, close, window_size)
GKYZV = GarmanKlassYangZhangVolatility(high, low, open, close, window_size)

In [ ]:
HV

,Close
Date,
2000-02-02 00:00:00-05:00,0.016190
2000-02-03 00:00:00-05:00,0.013915
2000-02-04 00:00:00-05:00,0.013915
2000-02-07 00:00:00-05:00,0.013916
2000-02-08 00:00:00-05:00,0.012880
...,...
2024-08-26 00:00:00-04:00,0.012825
2024-08-27 00:00:00-04:00,0.012825
2024-08-28 00:00:00-04:00,0.012852


In [ ]:
CCV

,0
Date,
2000-02-02 00:00:00-05:00,0.015897
2000-02-03 00:00:00-05:00,0.013595
2000-02-04 00:00:00-05:00,0.013590
2000-02-07 00:00:00-05:00,0.013590
2000-02-08 00:00:00-05:00,0.012572
...,...
2024-08-26 00:00:00-04:00,0.012568
2024-08-27 00:00:00-04:00,0.012570
2024-08-28 00:00:00-04:00,0.012593


In [ ]:
PV

,0
Date,
2000-02-01 00:00:00-05:00,0.014032
2000-02-02 00:00:00-05:00,0.013665
2000-02-03 00:00:00-05:00,0.012842
2000-02-04 00:00:00-05:00,0.012479
2000-02-07 00:00:00-05:00,0.012407
...,...
2024-08-26 00:00:00-04:00,0.008985
2024-08-27 00:00:00-04:00,0.008968
2024-08-28 00:00:00-04:00,0.008852


In [ ]:
GKV

,0
Date,
2000-02-01 00:00:00-05:00,0.014948
2000-02-02 00:00:00-05:00,0.014498
2000-02-03 00:00:00-05:00,0.013888
2000-02-04 00:00:00-05:00,0.013422
2000-02-07 00:00:00-05:00,0.013329
...,...
2024-08-26 00:00:00-04:00,0.009923
2024-08-27 00:00:00-04:00,0.009897
2024-08-28 00:00:00-04:00,0.009764


In [ ]:
RSV

,0
Date,
2000-02-01 00:00:00-05:00,0.012447
2000-02-02 00:00:00-05:00,0.011943
2000-02-03 00:00:00-05:00,0.012122
2000-02-04 00:00:00-05:00,0.011551
2000-02-07 00:00:00-05:00,0.011477
...,...
2024-08-26 00:00:00-04:00,0.009191
2024-08-27 00:00:00-04:00,0.009152
2024-08-28 00:00:00-04:00,0.009038


In [ ]:
GKYZV

,0
Date,
2000-02-02 00:00:00-05:00,0.012709
2000-02-03 00:00:00-05:00,0.012535
2000-02-04 00:00:00-05:00,0.012018
2000-02-07 00:00:00-05:00,0.011914
2000-02-08 00:00:00-05:00,0.011678
...,...
2024-08-26 00:00:00-04:00,0.011641
2024-08-27 00:00:00-04:00,0.011611
2024-08-28 00:00:00-04:00,0.011500


In [ ]:
start = datetime(1999, 12, 3)
end = datetime(2024, 9, 5)
tckr = '^GSPC'
ticker = yf.Ticker(tckr)
ticker_historical = ticker.history(start=start, end=end, interval="1d")
YZV = YangZhangVolatility(ticker_historical.High, ticker_historical.Low, ticker_historical.Open, ticker_historical.Close, window_size)

In [ ]:
YZV

,0
Date,
2000-02-02 00:00:00-05:00,0.101547
2000-02-03 00:00:00-05:00,0.102260
2000-02-04 00:00:00-05:00,0.099831
2000-02-07 00:00:00-05:00,0.099508
2000-02-08 00:00:00-05:00,0.098997
...,...
2024-08-28 00:00:00-04:00,0.088798
2024-08-29 00:00:00-04:00,0.089022
2024-08-30 00:00:00-04:00,0.086412


In [ ]:
YZV.drop(YZV.index[-2:], inplace=True)

In [ ]:
YZV

,0
Date,
2000-02-02 00:00:00-05:00,0.101547
2000-02-03 00:00:00-05:00,0.102260
2000-02-04 00:00:00-05:00,0.099831
2000-02-07 00:00:00-05:00,0.099508
2000-02-08 00:00:00-05:00,0.098997
...,...
2024-08-26 00:00:00-04:00,0.089535
2024-08-27 00:00:00-04:00,0.089347
2024-08-28 00:00:00-04:00,0.088798


In [ ]:
volatility_proxies = pd.DataFrame(index=HV.index)

In [ ]:
volatility_proxies['HV'] = HV.values
volatility_proxies['CCV'] = CCV.values
volatility_proxies['PV'] = PV.values[1:]
volatility_proxies['GKV'] = GKV.values[1:]
volatility_proxies['RSV'] = RSV.values[1:]
volatility_proxies['YZV'] = YZV.values
volatility_proxies['GKYZV'] = GKYZV.values

In [ ]:
HV.head()

,Close
Date,
2000-02-02 00:00:00-05:00,0.016190
2000-02-03 00:00:00-05:00,0.013915
2000-02-04 00:00:00-05:00,0.013915
2000-02-07 00:00:00-05:00,0.013916
2000-02-08 00:00:00-05:00,0.012880


In [ ]:
CCV.head()

,0
Date,
2000-02-02 00:00:00-05:00,0.015897
2000-02-03 00:00:00-05:00,0.013595
2000-02-04 00:00:00-05:00,0.013590
2000-02-07 00:00:00-05:00,0.013590
2000-02-08 00:00:00-05:00,0.012572


In [ ]:
PV.head()

,0
Date,
2000-02-01 00:00:00-05:00,0.014032
2000-02-02 00:00:00-05:00,0.013665
2000-02-03 00:00:00-05:00,0.012842
2000-02-04 00:00:00-05:00,0.012479
2000-02-07 00:00:00-05:00,0.012407


In [ ]:
GKV.head()

,0
Date,
2000-02-01 00:00:00-05:00,0.014948
2000-02-02 00:00:00-05:00,0.014498
2000-02-03 00:00:00-05:00,0.013888
2000-02-04 00:00:00-05:00,0.013422
2000-02-07 00:00:00-05:00,0.013329


In [ ]:
RSV.head()

,0
Date,
2000-02-01 00:00:00-05:00,0.012447
2000-02-02 00:00:00-05:00,0.011943
2000-02-03 00:00:00-05:00,0.012122
2000-02-04 00:00:00-05:00,0.011551
2000-02-07 00:00:00-05:00,0.011477


In [ ]:
YZV.head()

,0
Date,
2000-02-02 00:00:00-05:00,0.101547
2000-02-03 00:00:00-05:00,0.102260
2000-02-04 00:00:00-05:00,0.099831
2000-02-07 00:00:00-05:00,0.099508
2000-02-08 00:00:00-05:00,0.098997


In [ ]:
GKYZV.head()

,0
Date,
2000-02-02 00:00:00-05:00,0.012709
2000-02-03 00:00:00-05:00,0.012535
2000-02-04 00:00:00-05:00,0.012018
2000-02-07 00:00:00-05:00,0.011914
2000-02-08 00:00:00-05:00,0.011678


In [ ]:
volatility_proxies

,HV,CCV,PV,GKV,RSV,YZV,GKYZV
Date,,,,,,,
2000-02-02 00:00:00-05:00,0.016190,0.015897,0.013665,0.014498,0.011943,0.101547,0.012709
2000-02-03 00:00:00-05:00,0.013915,0.013595,0.012842,0.013888,0.012122,0.102260,0.012535
2000-02-04 00:00:00-05:00,0.013915,0.013590,0.012479,0.013422,0.011551,0.099831,0.012018
2000-02-07 00:00:00-05:00,0.013916,0.013590,0.012407,0.013329,0.011477,0.099508,0.011914
2000-02-08 00:00:00-05:00,0.012880,0.012572,0.011933,0.012918,0.011363,0.098997,0.011678
...,...,...,...,...,...,...,...
2024-08-26 00:00:00-04:00,0.012825,0.012568,0.008985,0.009923,0.009191,0.089535,0.011641
2024-08-27 00:00:00-04:00,0.012825,0.012570,0.008968,0.009897,0.009152,0.089347,0.011611
2024-08-28 00:00:00-04:00,0.012852,0.012593,0.008852,0.009764,0.009038,0.088798,0.011500


In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/'

Mounted at /content/drive


In [ ]:
filename = 'daily_volatility_proxies.csv'
volatility_proxies.to_csv(path + filename)